<a href="https://colab.research.google.com/github/mdeanne/NaNuParser/blob/master/NaNuParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ты должен загрузить файлы

1.   Нажать на стрелочку слева под кнопкой **+ Code**
2.   Вкладка **Files**
3.   **Upload** 
*    Файл с фамилиями должен называться names.xlsx
*    Остальные могут называться как угодно, главное, чтобы текст находился в первом столбце
4.   Нажать на кнопочку **Play** на ячейке ниже или сочетание клавиш **Shift + Enter**
5.   **Refrash** в левом меню
6.   В появившейся папке **output** можно скачать файлы через **ПКМ Download**
7.   Если запустить следующую ячейку, то будет создан заархивированный файл с результатами
8.   Если требуется запустить парсинг заново с другими файлами, а по одиночке удалять их лень, то нужно запустить последнюю ячейку удаляющую все и загрузить файлы заново

In [0]:
#@title Парсинг чисел и фамилий
import pandas as pd


def get_paths_data_files():
  file_names = !ls *.xlsx
  file_names = file_names[0].split()
  general_path = !pwd
  general_path = general_path[0] + '/'
  file_names.remove('names.xlsx')
  for i in range(len(file_names)):
    file_names[i] = general_path + file_names[i]
  return(file_names)

def read_file(parsing_path):
  df = pd.read_excel(parsing_path)
  return (df)

def read_names(parsing_path_names):
  df = pd.read_excel(parsing_path_names)
  return (df)

def num_extract(line):
  nums = set('0123456789')
  ret = 0
  list_rets = []
  for i in range(len(line)):
    if not nums.isdisjoint(line[i]):
      ret = ret * 10 + int(line[i])
  if ret != 0:
    list_rets.append(ret)
  ret = 0
  return (list_rets)

def make_time_lst(arr_line):
  lst = []
  for i in range(len(arr_line)):
    ret = num_extract(arr_line[i])
    if ret != 0:
      lst = lst + ret
  return (', '.join(str(x) for x in lst))

def name_ends_off(name):
  vovels = set('аоэеиыуёюя')
  i = 0
  while i < len(name) and not vovels.isdisjoint(name[-1 - i]):
    i = i + 1
  if i == 0:
    return(name)
  return(name[:-i])

def names_ends_off(name_list):
  ret = []
  for i in range(len(name_list)):
    ret.append(name_ends_off(name_list[i]).lower())
  return(ret)

def word_name_compare_indx(word, lst_names):
  
  for i in range(len(lst_names)):
    if lst_names[i] == word[:len(lst_names[i])].lower():
      return (i)
  return (-1)

def grab_names(lst_line, lst_names, original_names):
  ret = ""
  for i in range(len(lst_line)):
    tmp = word_name_compare_indx(lst_line[i], lst_names)
    if tmp != -1:
      ret = ret + ', ' + original_names[tmp]
  return (ret[2:])

def isNaN(num):
    return num != num

df_names = read_file('/content/names.xlsx')
original_names = df_names[list(df_names)[0]]
lst_names = names_ends_off(original_names)
files_paths = get_paths_data_files()
!mkdir output

for file_indx in range(len(files_paths)):
  if files_paths[file_indx] == '/content/names.xlsx':
    continue
  df = read_file(files_paths[file_indx])
  name_col = list(df)[0]
  lst_num = []
  lst_workers = []
  for i in range(len(df[name_col])):
    if isNaN(df[name_col][i]):
      continue
    lst_line = df[name_col][i].split()
    lst_num.append(make_time_lst(lst_line))
    lst_workers.append(grab_names(lst_line, lst_names, original_names))
  df['time'] = pd.Series(lst_num)
  df['workers'] = pd.Series(lst_workers)
  df.to_excel('/content/output/' + files_paths[file_indx][9:-5] + '_UPD.xlsx')


In [0]:
#@title Заархивировать результаты в файл output.zip
!zip -r output.zip output/

In [0]:
#@title Удалить все и быстро
!rm -rf *

NaNuParser (Parser of names and numbers) **Copyright (C)** 2019 <mdeanne> [mdeanne](https://github.com/mdeanne)